In [12]:
import pandas as pd
import matplotlib.pyplot as plt

#load the forecast files
df_prb = pd.read_csv("/content/DL_Prb_Utilization_Data.csv")
df_ue = pd.read_csv("/content/Avg_UE_Number_Data.csv")

#convert timestamp to datetime
df_prb["Timestamp"] = pd.to_datetime(df_prb["Timestamp"], format='mixed')
df_ue["Timestamp"] = pd.to_datetime(df_ue["Timestamp"], format='mixed')


#merge both KPI forecasts on Timestamp and nCI
print("PRB DataFrame columns:", df_prb.columns.tolist())
print("UE DataFrame columns:", df_ue.columns.tolist())

merged = pd.merge(df_prb, df_ue, on=["Timestamp", "NCI"], suffixes=("_prb", "_ue"))

#ask user for ES Mode
mode = input("Enter ES mode (Conservative / Moderate / Aggressive): ").capitalize()

#define thresholds for ES modes
thresholds = {
    "Conservative": {"DL_Prb_Utilization": 10, "Avg_UE_Number": 10},
    "Moderate": {"DL_Prb_Utilization": 15, "Avg_UE_Number": 13},
    "Aggressive": {"DL_Prb_Utilization": 20, "Avg_UE_Number": 17}
}

if mode not in thresholds:
    print("Invalid mode. Using Moderate.")
    mode = "Moderate"

#use thresholds
prb_thresh = thresholds[mode]["DL_Prb_Utilization"]
ue_thresh = thresholds[mode]["Avg_UE_Number"]

#filter data below threshold
filtered = merged[

   (merged["DL_Prb_Utilization"] <= prb_thresh) &
   (merged["Avg_UE_Number"] <= ue_thresh)

]
print("DL_Prb_Utilization Forecast Summary:")
print(merged["DL_Prb_Utilization"].describe())

print("\nAvg_UE_Number Forecast Summary:")
print(merged["Avg_UE_Number"].describe())
print(f"Filtered rows count: {len(filtered)}")

#final recommendations per overlapping window
window_recommendations = []

for cell in filtered["NCI"].unique():
    cell_data = filtered[filtered["NCI"] == cell].sort_values("Timestamp").copy()
    cell_data["Gap"] = cell_data["Timestamp"].diff().dt.total_seconds().div(60)
    cell_data["Group"] = (cell_data["Gap"].fillna(15).round() != 15).cumsum()


    for _, group in cell_data.groupby("Group"):
        if len(group) >= 2:  #minimum 2 intervals=30 mins
            start_time = group["Timestamp"].min()
            end_time = group["Timestamp"].max()
            duration = (end_time - start_time).seconds // 60 + 15  #inclusive of last interval
            if duration >= 30:
                window_recommendations.append({
                    "NCI": cell,
                    "Start_Time": start_time,
                    "End_Time": end_time,
                    "Duration_Minutes": duration,
                    "Recommendation": "Shut Down (Low Utilization)"
                })

#create DataFrame
rec_df = pd.DataFrame(window_recommendations)

#save to file with mode in filename
rec_df.to_csv(f"energy_saving_recommendations_{mode.lower()}.csv", index=False)
print(f"Saved detailed recommendations to energy_saving_recommendations_{mode.lower()}.csv")

PRB DataFrame columns: ['Timestamp', 'NCI', 'gNB', 'DL_Prb_Utilization']
UE DataFrame columns: ['Timestamp', 'NCI', 'gNB', 'Avg_UE_Number']
Enter ES mode (Conservative / Moderate / Aggressive): Aggressive
DL_Prb_Utilization Forecast Summary:
count    262656.000000
mean         46.049315
std          22.491132
min           7.000000
25%          30.000000
50%          44.000000
75%          61.000000
max          95.000000
Name: DL_Prb_Utilization, dtype: float64

Avg_UE_Number Forecast Summary:
count    262656.000000
mean         22.768770
std          14.114183
min           0.000000
25%          10.000000
50%          22.000000
75%          33.000000
max          60.000000
Name: Avg_UE_Number, dtype: float64
Filtered rows count: 38408
Saved detailed recommendations to energy_saving_recommendations_aggressive.csv
